This notebook is based on a Kaggle kernel [Toxic BERT plain vanila](https://www.kaggle.com/yuval6967/toxic-bert-plain-vanila) by [Yuval Reina](https://www.kaggle.com/yuval6967) with the following changes and additions:
- text preprocessing
- new binary loss function utilizing identities
- optimized trainig
- graduate accumulation
- new metrics

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np 
import pandas as pd

import datetime
import pkg_resources
import seaborn as sns
import time
import scipy.stats as stats
import gc
import re
import operator 
import sys
from sklearn import metrics
from sklearn import model_selection

import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from nltk.stem import PorterStemmer
from sklearn.metrics import roc_auc_score
%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
from apex import amp
import shutil

from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold

In [3]:
device=torch.device('cuda')

In [4]:
FOLD=0 # 0...3

MAX_SEQUENCE_LENGTH = 256
SEED = 1234
EPOCHS = 2
Data_dir="../input"
Input_dir = "../input"
WORK_DIR = "../output"
# num_to_load=1800000                        
# valid_size= 4874  
TOXICITY_COLUMN = 'target'

In [5]:
from pytorch_pretrained_bert import convert_tf_checkpoint_to_pytorch
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification,BertAdam

/home/akuritsyn/anaconda3/envs/fastai-tf/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, MutableMapping
/home/akuritsyn/anaconda3/envs/fastai-tf/lib/python3.7/site-packages/google/protobuf/descriptor.py:47: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from google.protobuf.pyext import _message
/home/akuritsyn/anaconda3/envs/fastai-tf/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216, got 192
  return f(*args, **kwds)
/home/akuritsyn/anaconda3/envs/fastai-tf/lib/python3.7/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back

In [6]:
BERT_MODEL_PATH = '../bert-pretrained-models/uncased_L-12_H-768_A-12/'

In [7]:
# # Translate model from tensorflow to pytorch

# convert_tf_checkpoint_to_pytorch.convert_tf_checkpoint_to_pytorch(
#     BERT_MODEL_PATH + 'bert_model.ckpt',
# BERT_MODEL_PATH + 'bert_config.json',
# WORK_DIR + 'pytorch_model.bin')

# shutil.copyfile(BERT_MODEL_PATH + 'bert_config.json', WORK_DIR + 'bert_config.json')

In [8]:
# This is the Bert configuration file
from pytorch_pretrained_bert import BertConfig

bert_config = BertConfig(BERT_MODEL_PATH+'bert_config.json')

In [ ]:
# Converting the lines to BERT format
# Thanks to https://www.kaggle.com/httpwwwfszyc/bert-in-keras-taming
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [12]:
%%time
tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
train_df_all = pd.read_csv(os.path.join(Data_dir,"train.csv"))   #.sample(num_to_load+valid_size,random_state=SEED)
print('loaded %d records' % len(train_df_all))

loaded 1804874 records
CPU times: user 13.8 s, sys: 924 ms, total: 14.7 s
Wall time: 14 s


In [1]:
#train_df_all.head().T

In [14]:
#train_df_all.sexual_explicit.unique()

In [18]:
# train_df_all=train_df_all[:1000000]
# print('working with %d records' % len(train_df_all))

In [19]:
%%time
# Make sure all comment_text values are strings
train_df_all['comment_text'] = train_df_all['comment_text'].astype(str) 

sequences = convert_lines(train_df_all["comment_text"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)

# with open(WORK_DIR+'/sequences.pkl', 'rb') as f:
#     sequences=pickle.load(f)


1512
CPU times: user 37min 39s, sys: 10.2 s, total: 37min 49s
Wall time: 37min 40s


In [ ]:
#with open(WORK_DIR+'/sequences.pkl', 'wb') as f:
#    pickle.dump(sequences, f)

In [20]:
train_df_all=train_df_all.fillna(0)
train_df_all = train_df_all.drop(['comment_text'],axis=1)

In [ ]:
sss = StratifiedKFold(n_splits=4, shuffle=True, random_state=SEED)
for i, (train_index, valid_index) in enumerate(sss.split(train_df_all[['male', 'female', 'homosexual_gay_or_lesbian', 'christian', 
            'jewish','muslim', 'black', 'white', 
            'psychiatric_or_mental_illness']], (train_df_all['target']>=0.5).astype(float))):
    
    if i != FOLD:
        continue
    
    X, X_val = sequences[train_index], sequences[valid_index]
    train_df = train_df_all.loc[train_index]
    test_df  = train_df_all.loc[valid_index]

In [21]:
# sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=SEED)
# for train_index, test_index in sss.split(train_df_all[['male', 'female', 'homosexual_gay_or_lesbian', 'christian', 
#             'jewish','muslim', 'black', 'white', 
#             'psychiatric_or_mental_illness']], (train_df_all['target']>=0.5).astype(float)):
#     X, X_val = sequences[train_index], sequences[test_index]
#     train_df = train_df_all.loc[train_index]
#     test_df  = train_df_all.loc[test_index]

In [22]:
#X = sequences[:num_to_load]
#X_val = sequences[-valid_size:]
# test_df=train_df.tail(valid_size).copy()
# train_df=train_df.head(num_to_load)

In [23]:
n_train = len(train_df); print(n_train)
n = len(test_df); print(n)

1624386
180488


In [24]:
# train_df=train_df[:int(n_train/2)] #1st half
# X=X[:int(n_train/2)]

In [25]:
del train_df_all

In [26]:
#validation
# convert target to 0,1
test_df['target']=(test_df['target']>=0.5).astype(float)
y_val = test_df['target'].values

In [27]:
# List all identities
identity_columns = [
    'male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish',
    'muslim', 'black', 'white', 'psychiatric_or_mental_illness']

y_aux_train = train_df[['target','severe_toxicity','obscene','identity_attack','insult','threat']]

In [28]:
y_aux_train.head()

,target,severe_toxicity,obscene,identity_attack,insult,threat
1069204,0.000000,0.0,0.0,0.0,0.000000,0.0
101718,0.000000,0.0,0.0,0.0,0.000000,0.0
1442426,0.000000,0.0,0.0,0.0,0.000000,0.0
268594,0.166667,0.0,0.0,0.0,0.166667,0.0
909947,0.000000,0.0,0.0,0.0,0.000000,0.0


In [30]:
def custom_loss(data, targets):
    ''' Define custom loss function for weighted BCE on 'target' column '''
    bce_loss_1 = nn.BCEWithLogitsLoss(weight=targets[:,1:2])(data[:,:1],targets[:,:1])
    bce_loss_2 = nn.BCEWithLogitsLoss()(data[:,1:],targets[:,2:])
    return (bce_loss_1 * loss_weight) + 50.*bce_loss_2

In [31]:
# Overall
weights = np.ones((len(train_df),)) / 4
# Subgroup
weights += (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) / 4
# Background Positive, Subgroup Negative
weights += (( (train_df['target'].values>=0.5).astype(bool).astype(np.int) +
   ((train_df[identity_columns].fillna(0).values<0.5)).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) *0.25
# Background Negative, Subgroup Positive
weights += (( (train_df['target'].values<0.5).astype(bool).astype(np.int) +
   (train_df[identity_columns].fillna(0).values>=0.5).sum(axis=1).astype(bool).astype(np.int) ) > 1 ).astype(bool).astype(np.int) *0.25
loss_weight = 1.0 / weights.mean()


In [32]:
#y_columns=['target']               
#y = train_df[y_columns].values[:num_to_load]               
#y_val = train_df[y_columns].values[num_to_load:]

# convert target to 0,1
train_df['target']=(train_df['target']>=0.5).astype(float)

y = np.vstack([train_df['target'].values,weights]).T

In [33]:
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(np.hstack([y, y_aux_train]),dtype=torch.float))

In [34]:
#input_model_file = "bert_14.bin"
output_model_file =  WORK_DIR+'/'+ f"bert_14_NewLoss_{FOLD}.bin"

lr=2e-5
batch_size = 32
accumulation_steps=2
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

model = BertForSequenceClassification.from_pretrained(WORK_DIR,cache_dir=None,num_labels=7)

#model = BertForSequenceClassification(bert_config, num_labels=7)
#model.load_state_dict(torch.load(input_model_file))

model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
train = train_dataset

num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=lr,
                     warmup=0.05,
                     t_total=num_train_optimization_steps)

model, optimizer = amp.initialize(model, optimizer, opt_level="O1",verbosity=0)
model=model.train()

tq = tqdm_notebook(range(EPOCHS))
for epoch in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0.
    avg_accuracy = 0.
    lossf=True #None
    tk0 = tqdm_notebook(enumerate(train_loader),total=len(train_loader),leave=False)
    optimizer.zero_grad()
    for i,(x_batch, y_batch) in tk0:
        y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
        loss = custom_loss(y_pred, y_batch.to(device))    #F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            optimizer.step()                            # Now we can do an optimizer step
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()
        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
    tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)


    torch.save(model.state_dict(), output_model_file+str(epoch))

In [35]:
# Run validation
# The following 2 lines are not needed but show how to download the model for prediction
#model = BertForSequenceClassification(bert_config,num_labels=7) #len(y_columns)
#model.load_state_dict(torch.load(output_model_file ))
#model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=32, shuffle=False)

tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)
    valid_preds[i*32:(i+1)*32]=pred[:,0].detach().cpu().squeeze().numpy()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): FusedLayerNorm(torch.Size([768]), eps=1e-12, eleme

In [36]:
# Correct metrics with >=0.5 and <0.5

def calculate_overall_auc(df, model_name):
    true_labels = df[TOXICITY_COLUMN]>=0.5
    predicted_labels = df[model_name]
    return metrics.roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)



SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return metrics.roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, model_name):
    subgroup_examples = df[df[subgroup]>=0.5]
    return compute_auc((subgroup_examples[label]>=0.5), subgroup_examples[model_name])

def compute_bpsn_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[(df[subgroup]>=0.5) & (df[label]<0.5)]
    non_subgroup_positive_examples = df[(df[subgroup]<0.5) & (df[label]>=0.5)]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label]>=0.5, examples[model_name])

def compute_bnsp_auc(df, subgroup, label, model_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[(df[subgroup]>=0.5) & (df[label]>=0.5)]
    non_subgroup_negative_examples = df[(df[subgroup]<0.5) & (df[label]<0.5)]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label]>=0.5, examples[model_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]>=0.5])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)


In [37]:
MODEL_NAME = 'model1'
test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
TOXICITY_COLUMN = 'target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'target')
bias_metrics_df
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))

,bnsp_auc,bpsn_auc,subgroup,subgroup_auc,subgroup_size
2,0.963049,0.900741,homosexual_gay_or_lesbian,0.862689,1155
6,0.968384,0.895131,black,0.875496,1464
7,0.968202,0.900900,white,0.882085,2471
5,0.962778,0.920467,muslim,0.888378,2123
4,0.962485,0.938364,jewish,0.915991,745
1,0.962441,0.955486,female,0.937961,5305
0,0.965690,0.951323,male,0.939194,4478
3,0.955646,0.965733,christian,0.945761,4145
8,0.980121,0.933677,psychiatric_or_mental_illness,0.953260,470


0.9429263571927614

In [38]:
calculate_overall_auc(test_df, MODEL_NAME)

0.9715708485519116